In [1]:
import cv2
import numpy as np

def BackgroundRemoval(cap, fgbg_params):
    fgbg = cv2.createBackgroundSubtractorKNN(**fgbg_params)

    while(1):
        # Take each frame
        ret, frame = cap.read()
        mask = fgbg.apply(frame)

        # Bitwise-AND mask and original image
        frame = cv2.bitwise_and(frame, frame, mask= mask)

        #cv2.imshow('frame',frame)
        #cv2.imshow('mask',mask)
        cv2.imshow('res',frame)

        #press 'q' to kill
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return 

# params for Background Removal
fgbg_params = dict( history=1000, 
                    dist2Threshold=1000, 
                    detectShadows=False )

cap = cv2.VideoCapture('Stereo_conveyor_without_occlusions.mp4')

BackgroundRemoval(cap, fgbg_params)

KeyboardInterrupt: 

In [3]:
import cv2
import numpy as np
import imutils

def BackgroundRemoval(cap, fgbg_params):
    fgbg = cv2.createBackgroundSubtractorKNN(**fgbg_params)
    
    while(1):
        # Take each frame
        ret, originalFrame = cap.read()
        
        if ret == False:
            break
        
        mask = fgbg.apply(originalFrame)
        
        kernel = np.ones((5,5), np.uint8) 
        
        mask = cv2.erode(mask, kernel, iterations=1) 
        mask = cv2.dilate(mask, kernel, iterations=1) 

        # Bitwise-AND mask and original image
        frame = cv2.bitwise_and(originalFrame, originalFrame, mask= mask)
        
        cropTop = 200
        cropBottom = 600
        cropLeft = 300
        cropRight = 1170
        cropped = mask[200:600, 300:1170]
        
        cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        #only proceed if at least one contour was found
        if len(cnts) > 0:
            # find the largest contour in the mask, then use
            # it to compute the minimum enclosing circle and
            # centroid
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            if M["m00"] == 0.0:
                continue
            center = (int(M["m10"] / M["m00"])+cropLeft, int(M["m01"] / M["m00"])+cropTop)
            
            # only proceed if the radius meets a minimum size
            if radius > 20 and radius < 200:
                # draw the circle and centroid on the frame,
                # then update the list of tracked points
                cv2.circle(originalFrame, (int(x)+cropLeft, int(y)+cropTop), int(radius),
                    (0, 255, 255), 2)
                cv2.circle(originalFrame, center, 5, (0, 0, 255), -1)
                
            text = "Object found at:"
            colour = (0, 255, 0)
            cv2.putText(originalFrame,"({}, {})".format(center[0], center[1]), (0, 100), cv2.FONT_ITALIC, 1, colour)
#             cv2.putText(originalFrame,"Hello World!!!", (0, 150), cv2.FONT_ITALIC, 2, colour)
            
        else: 
            text = "Object not found"
            colour = (0, 0, 255)
                
        cv2.putText(originalFrame, text, (0, 50), cv2.FONT_ITALIC, 1, colour)


        cv2.imshow('res',originalFrame)

        #press 'q' to kill
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return 

# params for Background Removal
fgbg_params = dict( history=1000, 
                    dist2Threshold=1000, 
                    detectShadows=False )

cap = cv2.VideoCapture('Stereo_conveyor_without_occlusions.mp4')

BackgroundRemoval(cap, fgbg_params)

In [2]:
import cv2
import numpy as np
import imutils


kalman = cv2.KalmanFilter(2, 1, 0)

kalman.transitionMatrix = np.array([[1., 1.], [0., 1.]])
kalman.measurementMatrix = 1. * np.ones((1, 2))
kalman.processNoiseCov = 1e-5 * np.eye(2)
kalman.measurementNoiseCov = 1e-1 * np.ones((1, 1))
kalman.errorCovPost = 1. * np.ones((2, 2))
kalman.statePost = 0.1 * np.random.randn(2, 1)

def BackgroundRemoval(cap, fgbg_params):
    fgbg = cv2.createBackgroundSubtractorKNN(**fgbg_params)
    
    while(1):
        # Take each frame
        ret, originalFrame = cap.read()
        
        prediction = kalman.predict()
        
        if ret == False:
            break
        
        mask = fgbg.apply(originalFrame)
        
        kernel = np.ones((5,5), np.uint8) 
        
        mask = cv2.erode(mask, kernel, iterations=1) 
        mask = cv2.dilate(mask, kernel, iterations=1) 

        # Bitwise-AND mask and original image
        frame = cv2.bitwise_and(originalFrame, originalFrame, mask= mask)
        
        cropTop = 200
        cropBottom = 600
        cropLeft = 300
        cropRight = 1170
        cropped = mask[200:600, 300:1170]
        
        cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        #only proceed if at least one contour was found
        if len(cnts) > 0:
            # find the largest contour in the mask, then use
            # it to compute the minimum enclosing circle and
            # centroid
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            if M["m00"] == 0.0:
                continue
            center = (int(M["m10"] / M["m00"])+cropLeft, int(M["m01"] / M["m00"])+cropTop)
            
            # only proceed if the radius meets a minimum size
            if radius > 20 and radius < 200:
                # draw the circle and centroid on the frame,
                # then update the list of tracked points
                cv2.circle(originalFrame, (int(x)+cropLeft, int(y)+cropTop), int(radius),
                    (0, 255, 255), 2)
                cv2.circle(originalFrame, center, 5, (0, 0, 255), -1)
                
            text = "Object found at:"
            colour = (0, 255, 0)
            cv2.putText(originalFrame,"({}, {})".format(center[0], center[1]), (0, 100), cv2.FONT_ITALIC, 1, colour)
#             cv2.putText(originalFrame,"Hello World!!!", (0, 150), cv2.FONT_ITALIC, 2, colour)
            
        else: 
            text = "Object not found"
            colour = (0, 0, 255)
                
        cv2.putText(originalFrame, text, (0, 50), cv2.FONT_ITALIC, 1, colour)


        cv2.imshow('res',originalFrame)

        #press 'q' to kill
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return 

# params for Background Removal
fgbg_params = dict( history=1000, 
                    dist2Threshold=1000, 
                    detectShadows=False )

cap = cv2.VideoCapture('Stereo_conveyor_without_occlusions.mp4')

BackgroundRemoval(cap, fgbg_params)